In [ ]:
"""This section defines a function to get the bus info given the number or the sswg name."""
import numpy as np
import pandas as pd

# Read the excel file
df = pd.read_excel("/Users/arthurli/Documents/Planning_Data_Dictionary_11022023_164400.xlsx",sheet_name=2,usecols="B,C,F,R")

# Define a function get the information of the bus
def get_bus_info(bus_number, sswg_name=None):
    """Input the bus number (int), return the SSWG bus name, bus voltage, and the planned longname"""

    bus = np.logical_or(df['SSWG BUS NUMBER'].values == bus_number,df['SSWG BUS NAME'].values == sswg_name)
    df_bus = df.loc[bus]
    try:
        if df_bus.iloc[0]['SSWG BASE KV'] % 1 == 0:
            return df_bus.iloc[0]['SSWG BUS NAME'], df_bus.iloc[0]['PLANNING BUS LONGNAME'].title(), int(df_bus.iloc[0]['SSWG BASE KV'])
        else:
            return df_bus.iloc[0]['SSWG BUS NAME'], df_bus.iloc[0]['PLANNING BUS LONGNAME'].title(), df_bus.iloc[0]['SSWG BASE KV']
    except IndexError:
        print(f'Bus {bus_number} is not in the planning data dictionary.')


In [ ]:
"""This section converts the bus number to long bus name for bus in report VOLTAGE violation table."""
import re

# User input the Bus description
bus = input("Please enter the bus in the report table: ")
bus = bus.rstrip()
bus = re.sub(r'\s+', ' ', bus)
bus = bus.rstrip(' kV')

# Test
#bus = '44343 IMPERL05T3_8 138'

# Get the bus number, sswg name and voltage
num = re.findall(r'(\d+) ', bus)[0]
sswg_name = re.findall(r' (\w+)', bus)[0]
volt = re.findall(r'\d+$',bus)[0]
try:
    [sswg_name, long_name, bus_voltage] = get_bus_info(int(num),sswg_name)
    if long_name[-2:] == 'kV':
        long_name = long_name[:-2]
        #print(f' the {long_name.title()} kV bus,')
        print(f'The {long_name.title()} kV bus begins to experience large voltage drop for the loss of ')
    else:
        #print(f' the {long_name.title()} ' + volt + ' kV bus,')
        print(f'The {long_name.title()} {bus_voltage} kV bus begins to experience large voltage drop for the loss of ')
except TypeError:
    print('Write this bus voltage violation manually to the report.')

In [ ]:
"""This section converts the bus number to long bus name for branch in report THERMAL violation table."""
import re

# User input the Branch description
branch = input("Please enter the branch in the report table: ")


# Test
# branch = '8186 KENEDYSW4A    138    8913 KARNES4A      138  1  '
# branch = '8186 KENEDYSW4A 138 8913 KARNES4A 138  1  '
# branch = '5400 SPRUCE 345 5295 MARTINEZ_5 345 1'          # Error because 5295 not in the planning data dictionary
# branch = '110015 WAP_WAP_G5 20.0 49216 GENWAP5_05_8 138 G1'
# branch = '49216 GENWAP5_05_8 138 44010 W_A_P__POI8A 138 GL'
# branch = '7244 L_CUERO_8_1Y 138 89450 THOMASTN4B 138 1'

branch = branch.rstrip()
branch = re.sub(r'\s+', ' ', branch)
branch = branch.replace('.0','')

# Get two bus numbers and voltage
info = re.findall(r'(\d+) \D', branch)[:3]
volt = re.findall(r' \d+ ',branch)
ckt = re.findall(r'\w+$',branch)[0]

bus_name = []
try:
    for i in range(2):
        [sswg_name, long_name, bus_voltage] = get_bus_info(int(info[i]),'')
        bus_name.append(long_name.title())

    if volt[0] == volt[1]:  # AC line
        print(f"The {bus_name[0]} to {bus_name[1]}"+ volt[0] +"kV line circuit " + ckt + ' begins to experience overloading for the loss of ')
    else:   # Transformer
        print(f"The {bus_name[0]}{volt[0]}kV to {bus_name[1]}"+ volt[1] +'kV transformer begins to experience overloading for the loss of ')
except TypeError:
    print('Write this branch thermal violation manually to the report.')

In [ ]:
"""This section is used to convert contingency to two different forms in reports. The year and season are needed."""
"""Run the previous section to get the all-together contigency file."""

import re

# User input to get the tear and season
#year = input("Please enter the year of your case (4 digits): ")
#season = input("Please enter the season of your case (3 letter): ")
#season = season .rstrip().upper()
year = '2029'
season = 'sum'
type = 1    # 1 or 2
season = season.upper()

# User input to get the contingency ID
id = input("Please enter the contingency ID you want to convert (DB_ID_XXXXX): ")
id = id.rstrip().upper()

"""Test different contingency."""
# id = 'DB_ID_7499'
# id = 'DB_ID_633'        # remove unit from bus
# id = 'DB_ID_26591'     # remove branch multiple lines
# id = 'DB_ID_38746'     # remove bus, remove branch combined

#id = 'DB_ID_12867'

contingency = ''

# Find the contingency in the all-together file
file_name = year[2:]+season+'_All.con'
with open (file_name) as file:
    lines = file.readlines()

    # looping through each line in the file 
    for line in lines: 
          
        # if line have the input string, get the index of that line  
        if '\''+id+'\'' in line: 
            idx = lines.index(line)
            

            # find where this contingency end
            while ' END' not in lines[idx]:
                contingency = contingency + lines[idx] #+ "\n"
                idx += 1
            break
    # closing file after reading 
    file.close() 

# Create the contingency for Table legend
cont_0 = contingency.splitlines()[1:]
cont_1 = contingency.splitlines()[1:]
cont_2 = contingency.splitlines()[1:]
for line in range(len(cont_1)):
    cont_0[line] = cont_0[line].split('/',1)[0].lstrip().rstrip()
    cont_1[line] = cont_1[line].split('/',1)[0].lstrip().rstrip().lower()
    cont_2[line] = cont_2[line].split('/',1)[0].lstrip().rstrip().lower()

    bus_list = re.findall(r'bus (\d+)', cont_1[line])
    for i in range(len(bus_list)):
        bus_number = int(bus_list[i])
        [sswg_name, long_name, bus_voltage] = get_bus_info(bus_number,'')
        if 'open bus' in cont_1[line]:
            cont_1[line] = cont_1[line].replace(f'{bus_list[i]}',f'{bus_list[i]} [{sswg_name} {bus_voltage}]')
        else:
            cont_1[line] = cont_1[line].replace(f'bus {bus_list[i]}',f'{bus_list[i]} [{sswg_name} {bus_voltage}]')
        if i == len(bus_list)-1:    # the last bus
            if len(bus_list) == 2:
                cont_2[line] = cont_2[line].replace(f'bus {bus_list[i]}',f'{long_name.title()} {bus_voltage} kV line')
            else:
                cont_2[line] = cont_2[line].replace(f'bus {bus_list[i]}',f'{long_name.title()} {bus_voltage} kV bus')
        else:
            cont_2[line] = cont_2[line].replace(f'bus {bus_list[i]}',f'{long_name.title()}')
        cont_2[line] = cont_2[line].replace('open branch from','the')
        cont_2[line] = cont_2[line].replace('Kv','kV')
        cont_2[line] = cont_2[line].replace('ckt','circuit')
        cont_2[line] = cont_2[line].replace('open','the')
        cont_2[line] = cont_2[line].replace('remove','the')

# Define a function to print out the contingency in lines
def print_con(cont):
    for line in range(len(cont)):
        print(cont[line])     

# Change cont2 from line lists to a sentence
cont_2_sen = ''
for i in range(len(cont_2)):
    if len(cont_2) != 1 and i == len(cont_2)-1:
        cont_2_sen += 'and '+cont_2[i] +','
    else:
        cont_2_sen = cont_2_sen + cont_2[i] + ', '

#print_con(cont_0)
if type == 1:
    print_con(cont_1)
else:
    print(cont_2_sen)